In [1]:
#Tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

#Pandas and numpy for data formats
import pandas as pd
import numpy as np


#glob for data import
import glob
import random
#PyTorch
import torch
import torch.nn as nn
#SK Learn
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
#from sklearn.utils import shuffle
import math
import sys
#MatPlotLib
import matplotlib.pyplot as plt

from models import simpleRegression
from models import feedForward, sequentialModel, sequentialDropout, lstmModel, cnnModel, cnnModelFilters, scoringSystem

#import functions
from dataImport import importTrainingData, importParamsData, importTestData, importTrainParameter, importTargetParameter, importEvalParameter
from preprocessing import preprocessData, createTrainTensors, createParamsTensors, preprocessParams, rescaleData, preprocessParameters


#importTrainData()
train_data = np.load("../train_40000.npy")
params_data = np.load("../train_target_40000.npy")
train_parameter = np.load("../train_parameter_40000.npy")
#
validation_data = np.load("../val_train_40000.npy")
validation_params = np.load("../val_target_40000.npy")
validation_parameter = np.load("../val_parameter_40000.npy")

#Create stats for zScore
mean_value_target = np.mean(params_data)
standard_deviation_target = np.std(params_data)

mean_value = np.mean(train_data)
standard_deviation = np.std(train_data)
measurements_to_remove = 30


median_train = False
replaceValuesBiggerOne_train=False
replaceFirstXValues_train=False
removeFirstXValues_train=True
normalizeData_train=False
zScore_train=True

train_data = preprocessData(train_data,
                          median=median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

train_target = preprocessData(params_data,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)

validation_data = preprocessData(validation_data,
                          median = median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

validation_target = preprocessData(validation_params,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)


number_of_measurements = 300
if removeFirstXValues_train == True:
    number_of_measurements = 300 - measurements_to_remove    
    
#reshaping train data
if type(train_data) != np.ndarray:
    train_data = train_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:
    train_data = train_data.reshape(-1, 55, number_of_measurements)

#reshaping target data
if type(params_data) != np.ndarray:
    params_data = params_data.to_numpy()

#reshaping val data
if type(validation_data) != np.ndarray:
    validation_data = validation_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:    
    validation_data = validation_data.reshape(-1, 55, number_of_measurements)

#reshaping val target
if type(validation_params) != np.ndarray:
    validation_params = validation_params.to_numpy()

    
def scaleToMaxToOne(dat, min, max):
    scaled_dat = dat / max
    return scaled_dat

def unscaleMaxToOne(scaled_dat, min, max):
    unscaled_dat = scaled_dat * max
    return unscaled_dat

min_target = np.min(train_target) 
max_target = np.max(train_target)
print(f"Rescaling targets with max = {max_target}")
train_target = scaleToMaxToOne(train_target, min_target, max_target)
#val_target = scaleToMaxToOne(validation_target, min_target, max_target)    
val_target = validation_target    


prevent_overfitting = keras.callbacks.EarlyStopping(monitor="val_loss", patience = 20, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_delta=0.0001)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Rescaling targets with max = 0.6713607926990712


In [2]:
def create_ANN_Dropout_Parameters(activation_function = 'selu'):
    model = Sequential()  

    inputData = tf.keras.Input(shape=(55, number_of_measurements))
    inputStellarFeatures = tf.keras.Input(shape=(6,))   

    weight_initializer = "glorot_uniform" # default is glorot_uniform

    n_units = 2048

  # 2D Dense layers
    hidden2D_1 = Dense(units=n_units, activation=activation_function, kernel_initializer=weight_initializer) (inputData)
  
    dropout1 = Dropout(rate = 0.2) (hidden2D_1)
  
    hidden2D_2 = Dense(units=n_units, activation=activation_function, kernel_initializer=weight_initializer) (dropout1)# (hidden2D_1)
    hidden2D_3 = Dense(units=n_units, activation=activation_function, kernel_initializer=weight_initializer) (hidden2D_2)
    hidden2D_4 = Dense(units=n_units, activation=activation_function, kernel_initializer=weight_initializer) (hidden2D_3)


    hidden2D_5 = Dense(units=n_units, activation=activation_function, kernel_initializer=weight_initializer) (hidden2D_4) 
    flattened2D = Flatten() (hidden2D_5) 
 
    activation_function_stellar = 'selu'

    hidden_Stellar_1 = Dense(units=10, activation=activation_function_stellar, kernel_initializer=weight_initializer) (inputStellarFeatures)
    hidden_Stellar_2 = Dense(units=10, activation=activation_function_stellar, kernel_initializer=weight_initializer) (hidden_Stellar_1)
    hidden_Stellar_3 = Dense(units=10, activation=activation_function_stellar, kernel_initializer=weight_initializer) (hidden_Stellar_2)
    concatLayer = tf.keras.layers.concatenate([flattened2D, hidden_Stellar_3]) 

    output = Dense(units=55, kernel_initializer=weight_initializer) (concatLayer)
    model = keras.Model(inputs=[inputData, inputStellarFeatures], outputs = [output])


    optim = Adam(learning_rate=0.0001) 
    model.compile(loss='mean_absolute_error', optimizer=optim)

    return model

model = create_ANN_Dropout_Parameters()
model2 = create_ANN_Dropout_Parameters()
model3 = create_ANN_Dropout_Parameters()
model4 = create_ANN_Dropout_Parameters()
model5 = create_ANN_Dropout_Parameters()

model_2 = create_ANN_Dropout_Parameters()
model2_2 = create_ANN_Dropout_Parameters()
model3_2 = create_ANN_Dropout_Parameters()
model4_2 = create_ANN_Dropout_Parameters()
model5_2 = create_ANN_Dropout_Parameters()

model_3 = create_ANN_Dropout_Parameters()
model2_3 = create_ANN_Dropout_Parameters()
model3_3 = create_ANN_Dropout_Parameters()
model4_3 = create_ANN_Dropout_Parameters()
model5_3 = create_ANN_Dropout_Parameters()

In [3]:
train_parameter = train_parameter.reshape(-1, 6)
validation_parameter = validation_parameter.reshape(-1, 6)

# Ensemble 1

## Model 1

In [4]:
history_test_1_1 = model.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 18s 14ms/step - loss: 0.3867 - val_loss: 0.1055
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1147 - val_loss: 0.0662
Epoch 3/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0766 - val_loss: 0.0522
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0589 - val_loss: 0.0438
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0474 - val_loss: 0.0342
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0387 - val_loss: 0.0274
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0326 - val_loss: 0.0243
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0276 - val_loss: 0.0234
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0237 - val_loss: 0.0185
Epoch 10/1000
1000/1000 [=====================

In [5]:
pred = model.predict((validation_data, validation_parameter))
pred_final = unscaleMaxToOne(pred, min_target, max_target)
score = scoringSystem(validation_target, pred_final)

print(score)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9695.009498875188, shape=(), dtype=float64)


In [6]:
last_five_idx = len(history_test_1_1.history["val_loss"])-5
val_loss_1 = sum(history_test_1_1.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1_1.history["loss"])-5
train_loss_1 = sum(history_test_1_1.history["loss"][last_five_idx:])/5

## Model 2

In [7]:
history_test_1_2 = model2.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 15s 14ms/step - loss: 0.5113 - val_loss: 0.1094
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1191 - val_loss: 0.0675
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0808 - val_loss: 0.0513
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0625 - val_loss: 0.0417
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0504 - val_loss: 0.0380
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0411 - val_loss: 0.0300
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0342 - val_loss: 0.0273
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0288 - val_loss: 0.0233
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0241 - val_loss: 0.0203
Epoch 10/1000
1000/1000 [=====================

In [8]:
pred2 = model2.predict((validation_data, validation_parameter))
pred_final2 = unscaleMaxToOne(pred2, min_target, max_target)
score2 = scoringSystem(validation_target, pred_final2)

print(score2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9671.606199675756, shape=(), dtype=float64)


In [9]:
last_five_idx = len(history_test_1_2.history["val_loss"])-5
val_loss_2 = sum(history_test_1_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1_2.history["loss"])-5
train_loss_2 = sum(history_test_1_2.history["loss"][last_five_idx:])/5

# Model 3

In [10]:
history_test_1_3 = model3.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 15s 14ms/step - loss: 0.3146 - val_loss: 0.0974
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1060 - val_loss: 0.0555
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0699 - val_loss: 0.0437
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0519 - val_loss: 0.0343
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0408 - val_loss: 0.0282
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0328 - val_loss: 0.0238
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0268 - val_loss: 0.0203
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0225 - val_loss: 0.0199
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0190 - val_loss: 0.0156
Epoch 10/1000
1000/1000 [=====================

In [11]:
pred3 = model3.predict((validation_data, validation_parameter))
pred_final3 = unscaleMaxToOne(pred3, min_target, max_target)
score3 = scoringSystem(validation_target, pred_final3)

print(score3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9705.20811154208, shape=(), dtype=float64)


In [12]:
last_five_idx = len(history_test_1_3.history["val_loss"])-5
val_loss_3 = sum(history_test_1_3.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1_3.history["loss"])-5
train_loss_3 = sum(history_test_1_3.history["loss"][last_five_idx:])/5

# Model 4

In [13]:
history_test_1_4 = model4.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5560 - val_loss: 0.1160
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1275 - val_loss: 0.0822
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0893 - val_loss: 0.0617
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0718 - val_loss: 0.0529
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0605 - val_loss: 0.0457
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0516 - val_loss: 0.0411
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0450 - val_loss: 0.0383
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0395 - val_loss: 0.0313
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0354 - val_loss: 0.0300
Epoch 10/1000
1000/1000 [=====================

In [14]:
pred4 = model4.predict((validation_data, validation_parameter))
pred_final4 = unscaleMaxToOne(pred4, min_target, max_target)
score4 = scoringSystem(validation_target, pred_final4)

print(score4)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9499.358799087355, shape=(), dtype=float64)


In [15]:
last_five_idx = len(history_test_1_4.history["val_loss"])-5
val_loss_4 = sum(history_test_1_4.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1_4.history["loss"])-5
train_loss_4 = sum(history_test_1_4.history["loss"][last_five_idx:])/5

# Model 5

In [16]:
history_test_1_5 = model5.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3851 - val_loss: 0.1037
Epoch 2/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1159 - val_loss: 0.0667
Epoch 3/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0791 - val_loss: 0.0503
Epoch 4/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0607 - val_loss: 0.0429
Epoch 5/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0497 - val_loss: 0.0345
Epoch 6/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0408 - val_loss: 0.0290
Epoch 7/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0346 - val_loss: 0.0264
Epoch 8/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0292 - val_loss: 0.0282
Epoch 9/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0259 - val_loss: 0.0213
Epoch 10/1000
1000/1000 [=====================

In [17]:
pred5 = model5.predict((validation_data, validation_parameter))
pred_final5 = unscaleMaxToOne(pred5, min_target, max_target)
score5 = scoringSystem(validation_target, pred_final5)

print(score5)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9729.480728718861, shape=(), dtype=float64)


In [18]:
last_five_idx = len(history_test_1_5.history["val_loss"])-5
val_loss_5 = sum(history_test_1_5.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1_5.history["loss"])-5
train_loss_5 = sum(history_test_1_5.history["loss"][last_five_idx:])/5

## Evaluation

In [19]:
pred_complete = pred_final + pred_final2 + pred_final3 + pred_final4 + pred_final5

In [20]:
pred_complete = pred_complete/5

In [21]:
score_complete = scoringSystem(validation_target, pred_complete)

print(score_complete)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9688.18700816499, shape=(), dtype=float64)


In [60]:
val_loss_complete = (val_loss_1 + val_loss_2 + val_loss_3 + val_loss_4 + val_loss_5)/5
print(val_loss_complete)

0.005839731078594923


# Esemble 2

## Model 1

In [23]:
history_test_1_2 = model_2.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4896 - val_loss: 0.1116
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1233 - val_loss: 0.0729
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0843 - val_loss: 0.0585
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0669 - val_loss: 0.0462
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0540 - val_loss: 0.0412
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0451 - val_loss: 0.0327
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0383 - val_loss: 0.0275
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0328 - val_loss: 0.0263
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0287 - val_loss: 0.0231
Epoch 10/1000
1000/1000 [=====================

In [24]:
pred_2 = model_2.predict((validation_data, validation_parameter))
pred_final_2 = unscaleMaxToOne(pred_2, min_target, max_target)
score_2 = scoringSystem(validation_target, pred_final_2)

print(score_2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9695.67259989907, shape=(), dtype=float64)


In [25]:
last_five_idx = len(history_test_1_2.history["val_loss"])-5
val_loss_1_2 = sum(history_test_1_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1_2.history["loss"])-5
train_loss_1_2 = sum(history_test_1_2.history["loss"][last_five_idx:])/5

## Model 2

In [26]:
history_test_2_2 = model2_2.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5883 - val_loss: 0.1274
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1291 - val_loss: 0.0815
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0904 - val_loss: 0.0571
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0720 - val_loss: 0.0520
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0606 - val_loss: 0.0419
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0518 - val_loss: 0.0392
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0453 - val_loss: 0.0368
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0402 - val_loss: 0.0351
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0363 - val_loss: 0.0307
Epoch 10/1000
1000/1000 [=====================

In [27]:
pred2_2 = model2_2.predict((validation_data, validation_parameter))
pred_final2_2 = unscaleMaxToOne(pred2_2, min_target, max_target)
score2_2 = scoringSystem(validation_target, pred_final2_2)

print(score2_2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9452.355834272086, shape=(), dtype=float64)


In [28]:
last_five_idx = len(history_test_2_2.history["val_loss"])-5
val_loss_2_2 = sum(history_test_2_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_2_2.history["loss"])-5
train_loss_2_2 = sum(history_test_2_2.history["loss"][last_five_idx:])/5

## Model 3

In [29]:
history_test_3_2 = model3_2.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3003 - val_loss: 0.0953
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1067 - val_loss: 0.0571
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0698 - val_loss: 0.0443
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0518 - val_loss: 0.0349
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0405 - val_loss: 0.0301
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0328 - val_loss: 0.0249
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0268 - val_loss: 0.0207
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0228 - val_loss: 0.0175
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0192 - val_loss: 0.0164
Epoch 10/1000
1000/1000 [=====================

In [30]:
pred3_2 = model3_2.predict((validation_data, validation_parameter))
pred_final3_2 = unscaleMaxToOne(pred3_2, min_target, max_target)
score3_2 = scoringSystem(validation_target, pred_final3_2)

print(score3_2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9685.414863583092, shape=(), dtype=float64)


In [31]:
last_five_idx = len(history_test_3_2.history["val_loss"])-5
val_loss_3_2 = sum(history_test_3_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_3_2.history["loss"])-5
train_loss_3_2 = sum(history_test_3_2.history["loss"][last_five_idx:])/5

## Model 4

In [32]:
history_test_4_2 = model4_2.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 15s 14ms/step - loss: 0.3136 - val_loss: 0.0949
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1076 - val_loss: 0.0580
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0716 - val_loss: 0.0468
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0540 - val_loss: 0.0343
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0422 - val_loss: 0.0283
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0343 - val_loss: 0.0245
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0279 - val_loss: 0.0209
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0235 - val_loss: 0.0180
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0200 - val_loss: 0.0165
Epoch 10/1000
1000/1000 [=====================

In [33]:
pred4_2 = model4_2.predict((validation_data, validation_parameter))
pred_final4_2 = unscaleMaxToOne(pred4_2, min_target, max_target)
score4_2 = scoringSystem(validation_target, pred_final4_2)

print(score4_2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9640.503476479429, shape=(), dtype=float64)


In [34]:
last_five_idx = len(history_test_4_2.history["val_loss"])-5
val_loss_4_2 = sum(history_test_4_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_4_2.history["loss"])-5
train_loss_4_2 = sum(history_test_4_2.history["loss"][last_five_idx:])/5

## Model 5

In [35]:
history_test_5_2 = model5_2.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4153 - val_loss: 0.1129
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1225 - val_loss: 0.0797
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0832 - val_loss: 0.0562
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0662 - val_loss: 0.0496
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0540 - val_loss: 0.0435
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0458 - val_loss: 0.0379
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0395 - val_loss: 0.0315
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0343 - val_loss: 0.0265
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0300 - val_loss: 0.0295
Epoch 10/1000
1000/1000 [=====================

In [36]:
pred5_2 = model5_2.predict((validation_data, validation_parameter))
pred_final5_2 = unscaleMaxToOne(pred5_2, min_target, max_target)
score5_2 = scoringSystem(validation_target, pred_final5_2)

print(score5_2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9700.107393932603, shape=(), dtype=float64)


In [38]:
last_five_idx = len(history_test_5_2.history["val_loss"])-5
val_loss_5_2 = sum(history_test_5_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_5_2.history["loss"])-5
train_loss_5_2 = sum(history_test_5_2.history["loss"][last_five_idx:])/5

In [39]:
last_five_idx = len(history_test_4_2.history["val_loss"])-5
val_loss_4_2 = sum(history_test_4_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_4_2.history["loss"])-5
train_loss_4_2 = sum(history_test_4_2.history["loss"][last_five_idx:])/5

## Evaluation

In [59]:
pred_complete_2 = pred_final_2 + pred_final2_2 + pred_final3_2 + pred_final4_2 + pred_final5_2
pred_complete_2 = pred_complete_2/5
score_complete_2 = scoringSystem(validation_target, pred_complete_2)

print(score_complete_2)

val_loss_complete_2 = (val_loss_1_2 + val_loss_2_2 + val_loss_3_2 + val_loss_4_2 + val_loss_5_2)/5
print(val_loss_complete_2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9656.30338854739, shape=(), dtype=float64)
0.006093459986150265


# Esemble 3

## Model 1

In [41]:
history_test_1_3 = model_3.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 15s 14ms/step - loss: 0.5050 - val_loss: 0.1053
Epoch 2/1000
1000/1000 [==============================] - 17s 17ms/step - loss: 0.1248 - val_loss: 0.0712
Epoch 3/1000
1000/1000 [==============================] - 16s 16ms/step - loss: 0.0854 - val_loss: 0.0652
Epoch 4/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0683 - val_loss: 0.0448
Epoch 5/1000
1000/1000 [==============================] - 16s 16ms/step - loss: 0.0561 - val_loss: 0.0427
Epoch 6/1000
1000/1000 [==============================] - 18s 18ms/step - loss: 0.0468 - val_loss: 0.0360
Epoch 7/1000
1000/1000 [==============================] - 17s 17ms/step - loss: 0.0395 - val_loss: 0.0371
Epoch 8/1000
1000/1000 [==============================] - 17s 17ms/step - loss: 0.0342 - val_loss: 0.0286
Epoch 9/1000
1000/1000 [==============================] - 16s 16ms/step - loss: 0.0300 - val_loss: 0.0238
Epoch 10/1000
1000/1000 [=====================

In [42]:
pred_3 = model_3.predict((validation_data, validation_parameter))
pred_final_3 = unscaleMaxToOne(pred_3, min_target, max_target)
score_3 = scoringSystem(validation_target, pred_final_3)

print(score_3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9708.030676459035, shape=(), dtype=float64)


In [43]:
last_five_idx = len(history_test_1_3.history["val_loss"])-5
val_loss_1_3 = sum(history_test_1_3.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1_3.history["loss"])-5
train_loss_1_3 = sum(history_test_1_3.history["loss"][last_five_idx:])/5

## Model 2

In [44]:
history_test_2_3 = model2_3.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 15s 14ms/step - loss: 0.3860 - val_loss: 0.1061
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1163 - val_loss: 0.0660
Epoch 3/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0791 - val_loss: 0.0538
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0604 - val_loss: 0.0397
Epoch 5/1000
1000/1000 [==============================] - 15s 16ms/step - loss: 0.0484 - val_loss: 0.0371
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0401 - val_loss: 0.0311
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0337 - val_loss: 0.0250
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0291 - val_loss: 0.0232
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0245 - val_loss: 0.0216
Epoch 10/1000
1000/1000 [=====================

In [45]:
pred2_3 = model2_3.predict((validation_data, validation_parameter))
pred_final2_3 = unscaleMaxToOne(pred2_3, min_target, max_target)
score2_3 = scoringSystem(validation_target, pred_final2_3)

print(score2_3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9709.512188851068, shape=(), dtype=float64)


In [46]:
last_five_idx = len(history_test_2_3.history["val_loss"])-5
val_loss_2_3 = sum(history_test_2_3.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_2_3.history["loss"])-5
train_loss_2_3 = sum(history_test_2_3.history["loss"][last_five_idx:])/5

## Model 3

In [47]:
history_test_3_3 = model3_3.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 17s 15ms/step - loss: 0.8963 - val_loss: 0.1229
Epoch 2/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1343 - val_loss: 0.0791
Epoch 3/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0920 - val_loss: 0.0672
Epoch 4/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0743 - val_loss: 0.0568
Epoch 5/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0628 - val_loss: 0.0456
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0537 - val_loss: 0.0419
Epoch 7/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0461 - val_loss: 0.0383
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0406 - val_loss: 0.0324
Epoch 9/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0354 - val_loss: 0.0300
Epoch 10/1000
1000/1000 [=====================

In [48]:
pred3_3 = model3_3.predict((validation_data, validation_parameter))
pred_final3_3 = unscaleMaxToOne(pred3_3, min_target, max_target)
score3_3 = scoringSystem(validation_target, pred_final3_3)

print(score3_3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9701.401859294267, shape=(), dtype=float64)


In [49]:
last_five_idx = len(history_test_3_3.history["val_loss"])-5
val_loss_3_3 = sum(history_test_3_3.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_3_3.history["loss"])-5
train_loss_3_3 = sum(history_test_3_3.history["loss"][last_five_idx:])/5

## Model 4

In [50]:
history_test_4_3 = model4_3.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.5585 - val_loss: 0.1388
Epoch 2/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.1362 - val_loss: 0.0857
Epoch 3/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0973 - val_loss: 0.0704
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0796 - val_loss: 0.0667
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0679 - val_loss: 0.0498
Epoch 6/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0595 - val_loss: 0.0524
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0522 - val_loss: 0.0427
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0463 - val_loss: 0.0385
Epoch 9/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0414 - val_loss: 0.0399
Epoch 10/1000
1000/1000 [=====================

In [51]:
pred4_3 = model4_3.predict((validation_data, validation_parameter))
pred_final4_3 = unscaleMaxToOne(pred4_3, min_target, max_target)
score4_3 = scoringSystem(validation_target, pred_final4_3)

print(score4_3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9603.650220422618, shape=(), dtype=float64)


In [52]:
last_five_idx = len(history_test_4_3.history["val_loss"])-5
val_loss_4_3 = sum(history_test_4_3.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_4_3.history["loss"])-5
train_loss_4_3 = sum(history_test_4_3.history["loss"][last_five_idx:])/5

## Model 5

In [53]:
history_test_5_3 = model5_3.fit((train_data, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.5084 - val_loss: 0.1090
Epoch 2/1000
1000/1000 [==============================] - 16s 16ms/step - loss: 0.1213 - val_loss: 0.0673
Epoch 3/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0833 - val_loss: 0.0554
Epoch 4/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0650 - val_loss: 0.0459
Epoch 5/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0534 - val_loss: 0.0408
Epoch 6/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0450 - val_loss: 0.0343
Epoch 7/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0378 - val_loss: 0.0334
Epoch 8/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0330 - val_loss: 0.0276
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0292 - val_loss: 0.0230
Epoch 10/1000
1000/1000 [=====================

In [54]:
pred5_3 = model5_3.predict((validation_data, validation_parameter))
pred_final5_3 = unscaleMaxToOne(pred5_3, min_target, max_target)
score5_3 = scoringSystem(validation_target, pred_final5_3)

print(score5_3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9547.581248130524, shape=(), dtype=float64)


In [55]:
last_five_idx = len(history_test_5_3.history["val_loss"])-5
val_loss_5_3 = sum(history_test_5_3.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_5_3.history["loss"])-5
train_loss_5_3 = sum(history_test_5_3.history["loss"][last_five_idx:])/5

## Evaluation

In [62]:
pred_complete_3 = pred_final_3 + pred_final2_3 + pred_final3_3 + pred_final4_3 + pred_final5_3
pred_complete_3 = pred_complete_3/5
score_complete_3 = scoringSystem(validation_target, pred_complete_3)

print(score_complete_3)

val_loss_complete_3 = (val_loss_1_3 + val_loss_2_3 + val_loss_3_3 + val_loss_4_3 + val_loss_5_3)/5
print(val_loss_complete_3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9681.846135398791, shape=(), dtype=float64)
0.006417927350848914


# Complete Evaluation

In [63]:
score_complete_final = (score_complete + score_complete_2 + score_complete_3)/3
val_loss_complete_final = (val_loss_complete + val_loss_complete_2 + val_loss_complete_3)/3

print(score_complete_final)
print(val_loss_complete_final)

tf.Tensor(9675.445510703725, shape=(), dtype=float64)
0.0061170394718647
